# Table 4. Association of measurements and treatments with outcomes in patients with on-scene survival
-	NACA and hospital survival
-	ISS and Analgetics
-	Sex and Analgetics
-	Tracheal Intubation and Survival
-	NACA and i.v. access / i.o.
-	NACA and ISS
-	overtriage? NACA ≥ 4 vs ISS >= 23
-	MECHANISM and NACA/ISS





In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import spearmanr


In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/pediatric_trauma/data/Data_PedRegaTrauma_coded_for_analysis_250417.xlsx'

In [ ]:
df = pd.read_excel(data_path, sheet_name='All centres cleaned')

## NACA and hospital survival

In [ ]:
df.NACA.unique()

In [ ]:
df.NACA.value_counts()

In [ ]:
# logistic regression with hostpital survival as outcome and NACA as predictor

# map Place discharge to hospital survival (1-4): 1 / 0: 0 / 9999: nan
df['hospital_survival'] = df['Place discharge'].apply(lambda x: 1 if x in [1, 2, 3, 4] else (0 if x == 0 else pd.NA))
temp_df = df[['NACA', 'hospital_survival']].copy()
temp_df.dropna(inplace=True)
# this excluedes naca 7 as no hospital survival data is available for it

temp_df['NACA'] = temp_df['NACA'].astype(int)
temp_df['hospital_survival'] = temp_df['hospital_survival'].astype(int)

survival_logit_model = sm.Logit(temp_df['hospital_survival'], sm.add_constant(temp_df['NACA']))
survival_logit_result = survival_logit_model.fit()
print(survival_logit_result.summary())

In [ ]:
# compute odds ratio for NACA, with 95% CI
import numpy as np
naca_coef = survival_logit_result.params['NACA']
naca_ci_low, naca_ci_high = survival_logit_result.conf_int().loc['NACA']
NACA_survival_OR = np.exp(naca_coef)
NACA_survival_OR_ci = (np.exp(naca_ci_low), np.exp(naca_ci_high))
print(f"NACA OR: {NACA_survival_OR:.3f} (95% CI {NACA_survival_OR_ci[0]:.3f}–{NACA_survival_OR_ci[1]:.3f}, pval={survival_logit_result.pvalues['NACA']})")

In [ ]:
# for every NACA cut-off from 0 to 7, compute sensitivity and specificity for hospital mortality
for naca_cutoff in range(0, 8):
    temp_df['predicted_mortality'] = temp_df['NACA'].apply(lambda x: 1 if x >= naca_cutoff else 0)
    tp = ((temp_df['predicted_mortality'] == 1) & (temp_df['hospital_survival'] == 0)).sum()
    tn = ((temp_df['predicted_mortality'] == 0) & (temp_df['hospital_survival'] == 1)).sum()
    fp = ((temp_df['predicted_mortality'] == 1) & (temp_df['hospital_survival'] == 1)).sum()
    fn = ((temp_df['predicted_mortality'] == 0) & (temp_df['hospital_survival'] == 0)).sum()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f'NACA >= {naca_cutoff}, Sensitivity: {sensitivity:.3f}, Specificity: {specificity:.3f}')

In [ ]:
# ROC curve NACA predicting hospital mortality
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(-1*temp_df['hospital_survival'] + 1, temp_df['NACA'])
roc_auc = auc(fpr, tpr)
import matplotlib.pyplot as plt
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic - NACA predicting hospital survival')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np
youden_index = tpr - fpr
optimal_idx = np.argmax(youden_index)
optimal_threshold = thresholds[optimal_idx]
print(f'Optimal NACA threshold for hospital survival: {optimal_threshold}')

# NACA and LOS

In [ ]:
df['Length of hospital stay'] = df['Length of hospital stay'].replace(999, pd.NA)
df['length of ICU stay'] = df['length of ICU stay'].replace(999, pd.NA)
# to numeric
df['Length of hospital stay'] = pd.to_numeric(df['Length of hospital stay'], errors='coerce')
df['length of ICU stay'] = pd.to_numeric(df['length of ICU stay'], errors='coerce')

# linear regression with length of hospital stay as outcome and NACA as predictor
temp_df = df[['NACA', 'Length of hospital stay']].copy()
temp_df.dropna(inplace=True)
temp_df['NACA'] = temp_df['NACA'].astype(int)
temp_df['Length of hospital stay'] = temp_df['Length of hospital stay'].astype(float)
length_lm_model = sm.OLS(temp_df['Length of hospital stay'], sm.add_constant(temp_df['NACA']))
length_lm_result = length_lm_model.fit()
print(length_lm_result.summary())

In [ ]:
#  regplot
import seaborn as sns
import matplotlib.pyplot as plt

sns.swarmplot(x='NACA', y='Length of hospital stay', data=temp_df)

## ISS and Analgetics

In [ ]:
# any analgetics and ISS

df['In-hospital Injury Severity Score (ISS)'].replace(999, pd.NA, inplace=True)
df['ISS'] = df['In-hospital Injury Severity Score (ISS)'].apply(lambda x: x.split('/')[0].strip() if isinstance(x, str) else x)
df['ISS'] = df['ISS'].apply(lambda x: int(x.split(' ')[0].strip()) if isinstance(x, str) else x)
df['ISS'] = pd.to_numeric(df['ISS'])

df['any_analgetics'] = (df['Analgetics'] != 0).astype(int)
temp_df = df[['ISS', 'any_analgetics']].copy()
temp_df.dropna(inplace=True)
temp_df['ISS'] = temp_df['ISS'].astype(int)
iss_analgetics_logit_model = sm.Logit(temp_df['any_analgetics'], sm.add_constant(temp_df['ISS']))
iss_analgetics_logit_result = iss_analgetics_logit_model.fit()
print(iss_analgetics_logit_result.summary())
OR_iss_analgetics = np.exp(iss_analgetics_logit_result.params['ISS'])
ci_low, ci_high = iss_analgetics_logit_result.conf_int().loc['ISS']
OR_iss_analgetics_ci = (np.exp(ci_low), np.exp(ci_high))
print(f"ISS OR for any analgetics: {OR_iss_analgetics:.3f} (95% CI {OR_iss_analgetics_ci[0]:.3f}–{OR_iss_analgetics_ci[1]:.3f}, pval={iss_analgetics_logit_result.pvalues['ISS']})")

In [ ]:
# analgetics : analgetic_id: analgetic dose; analgetic_id: analgetic dose; ... (analgetic_id 1: fentanyl, 2: ketamine)
df['fentanyl_dose'] = 0
df['ketamine_dose'] = 0
df['Analgetics'] = df['Analgetics'].str.replace(',', ';')  # replace commas with semicolons for consistency
for i, row in df.iterrows():
    if pd.isna(row['Analgetics']) or row['Analgetics'] == 0:
        continue
    for analgetic in row['Analgetics'].split(';'):
        if analgetic.strip() == '':
            continue
        analgetic_id, dose = analgetic.split(':')
        
        analgetic_id = int(analgetic_id.strip())
        # remove mcg or mg from dose
        dose = dose.replace('mcg', '').replace('mg', '').strip()
        dose = float(dose) if dose else 0
        if analgetic_id == 1:
            df.at[i, 'fentanyl_dose'] = dose
        elif analgetic_id == 2:
            df.at[i, 'ketamine_dose'] = dose

In [ ]:
df['Bodyweight (kg)'] = df['Bodyweight (kg)'].astype(str).str.replace(',', '.')
df['Bodyweight (kg)'].replace('nan', pd.NA, inplace=True)
df['fentanyl_dose_norm'] = df['fentanyl_dose'] / pd.to_numeric(df['Bodyweight (kg)'])
df['ketamine_dose_norm'] = df['ketamine_dose'] / pd.to_numeric(df['Bodyweight (kg)'])

In [ ]:
# ISS as predictor for fentanyl/ketamine administration
temp_df = df[['ISS', 'fentanyl_dose', 'ketamine_dose']].copy()
temp_df['fentanyl_binary'] = (temp_df['fentanyl_dose'] > 0).astype(int)
temp_df['ketamine_binary'] = (temp_df['ketamine_dose'] > 0).astype(int)
temp_df.dropna(inplace=True)
temp_df['ISS'] = temp_df['ISS'].astype(int)
fentanyl_iss_logit_model = sm.Logit(temp_df['fentanyl_binary'], sm.add_constant(temp_df['ISS']))
fentanyl_iss_logit_result = fentanyl_iss_logit_model.fit()
print(fentanyl_iss_logit_result.summary())
ketamine_iss_logit_model = sm.Logit(temp_df['ketamine_binary'], sm.add_constant(temp_df['ISS']))
ketamine_iss_logit_result = ketamine_iss_logit_model.fit()
print(ketamine_iss_logit_result.summary())

In [ ]:
# associate fentanyl and ketamine doses with ISS
temp_df = df[['ISS', 'fentanyl_dose_norm', 'ketamine_dose_norm']].copy()
temp_df.dropna(inplace=True)
temp_df['ISS'] = temp_df['ISS'].astype(int)
fentanyl_iss_model = sm.OLS(temp_df['fentanyl_dose_norm'], sm.add_constant(temp_df['ISS']))
fentanyl_iss_result = fentanyl_iss_model.fit()
print(fentanyl_iss_result.summary())
ketamine_iss_model = sm.OLS(temp_df['ketamine_dose_norm'], sm.add_constant(temp_df['ISS']))
ketamine_iss_result = ketamine_iss_model.fit()
print(ketamine_iss_result.summary())

In [ ]:
# plot fentanyl and ketamine doses against ISS
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
# sns.scatterplot(data=df, x='ISS', y='fentanyl_dose_norm', label='Fentanyl Dose', color='blue')
# sns.scatterplot(data=df, x='ISS', y='ketamine_dose_norm', label='Ketamine Dose', color='orange')

sns.regplot(data=df, x='ISS', y='fentanyl_dose_norm', scatter=True, color='blue', label='Fentanyl Regression Line')
sns.regplot(data=df, x='ISS', y='ketamine_dose_norm', scatter=True, color='orange', label='Ketamine Regression Line')

plt.title('Fentanyl and Ketamine Doses vs ISS')
plt.xlabel('In-hospital Injury Severity Score (ISS)')
plt.ylabel('Analgetic Dose (mcg or mg) per kg)')
plt.legend()
plt.show()

In [ ]:
# make 2 plots depending on intubation status - df['Intubation (y/n)']
ax = sns.FacetGrid(df, col='Intubation (y/n)', height=6, aspect=1.2)
ax.map(sns.regplot, 'ISS', 'fentanyl_dose_norm', scatter=True, color='blue', label='Fentanyl Regression Line', scatter_kws={'alpha':0.3})
ax.map(sns.regplot, 'ISS', 'ketamine_dose_norm', scatter=True, color='orange', label='Ketamine Regression Line', scatter_kws={'alpha':0.3})
ax.set_axis_labels('In-hospital Injury Severity Score (ISS)', 'Analgetic Dose (mcg or mg) per kg)')
ax.set_titles(col_template='Intubation Status: {col_name}')
ax.add_legend()
plt.subplots_adjust(top=0.85)
ax.fig.suptitle('Fentanyl and Ketamine Doses vs ISS by Intubation Status', fontsize=16)
plt.show()

In [ ]:
df['Intubation (y/n)'].unique()

In [ ]:
# only non intubated patients
non_intubated_df = df[df['Intubation (y/n)'] == 0]
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.regplot(data=non_intubated_df, x='ISS', y='fentanyl_dose_norm', scatter=True, color='blue', label='Fentanyl Regression Line', scatter_kws={'alpha':0.3})
ax = sns.regplot(data=non_intubated_df, x='ISS', y='ketamine_dose_norm', scatter=True, color='orange', label='Ketamine Regression Line', scatter_kws={'alpha':0.3})
# ax.set_title('Fentanyl and Ketamine Doses vs ISS (Non-Intubated Patients)', fontsize=16)
# ax.set_axis_labels('In-hospital Injury Severity Score (ISS)', 'Analgetic Dose (mcg or mg) per kg)')
ax.set_xlabel('In-hospital Injury Severity Score (ISS)')
ax.set_ylabel('Analgetic Dose (mcg or mg) per kg)')


In [ ]:
# fig.savefig('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/figures/analgetics_vs_iss_non_intubated.png', bbox_inches='tight', dpi=600)

In [ ]:
# association of fentanyl and ketamine doses with ISS in intubated and non-intubated patients
intubated_df = df[df['Intubation (y/n)'] == 1]
non_intubated_df = df[df['Intubation (y/n)'] == 0]

print(f"Intubated patients: {len(intubated_df)}")
intubated_temp_df = intubated_df[['ISS', 'fentanyl_dose_norm', 'ketamine_dose_norm']].copy()
intubated_temp_df.dropna(inplace=True)
intubated_temp_df['ISS'] = intubated_temp_df['ISS'].astype(int)
fentanyl_intubated_model = sm.OLS(intubated_temp_df['fentanyl_dose_norm'], sm.add_constant(intubated_temp_df['ISS']))
fentanyl_intubated_result = fentanyl_intubated_model.fit()
print(fentanyl_intubated_result.summary())
ketamine_intubated_model = sm.OLS(intubated_temp_df['ketamine_dose_norm'], sm.add_constant(intubated_temp_df['ISS']))
ketamine_intubated_result = ketamine_intubated_model.fit()
print(ketamine_intubated_result.summary())

print(f"Non-intubated patients: {len(non_intubated_df)}")
non_intubated_temp_df = non_intubated_df[['ISS', 'fentanyl_dose_norm', 'ketamine_dose_norm']].copy()
non_intubated_temp_df.dropna(inplace=True)
non_intubated_temp_df['ISS'] = non_intubated_temp_df['ISS'].astype(int)
fentanyl_non_intubated_model = sm.OLS(non_intubated_temp_df['fentanyl_dose_norm'], sm.add_constant(non_intubated_temp_df['ISS']))
fentanyl_non_intubated_result = fentanyl_non_intubated_model.fit()
print(fentanyl_non_intubated_result.summary())
ketamine_non_intubated_model = sm.OLS(non_intubated_temp_df['ketamine_dose_norm'], sm.add_constant(non_intubated_temp_df['ISS']))
ketamine_non_intubated_result = ketamine_non_intubated_model.fit()
print(ketamine_non_intubated_result.summary())



In [ ]:
# ISS as predictor for fentanyl/ketamine administration in intubated and non-intubated patients
intubated_temp_df['fentanyl_binary'] = (intubated_temp_df['fentanyl_dose_norm'] > 0).astype(int)
intubated_temp_df['ketamine_binary'] = (intubated_temp_df['ketamine_dose_norm'] > 0).astype(int)
intubated_temp_df.dropna(inplace=True)
intubated_temp_df['ISS'] = intubated_temp_df['ISS'].astype(int)

print("Intubated patients logistic regression:")
fentanyl_intubated_iss_logit_model = sm.Logit(intubated_temp_df['fentanyl_binary'], sm.add_constant(intubated_temp_df['ISS']))
fentanyl_intubated_iss_logit_result = fentanyl_intubated_iss_logit_model.fit()
print(fentanyl_intubated_iss_logit_result.summary())
ketamine_intubated_iss_logit_model = sm.Logit(intubated_temp_df['ketamine_binary'], sm.add_constant(intubated_temp_df['ISS']))
ketamine_intubated_iss_logit_result = ketamine_intubated_iss_logit_model.fit()
print(ketamine_intubated_iss_logit_result.summary())

print("Non-intubated patients logistic regression:")
non_intubated_temp_df['fentanyl_binary'] = (non_intubated_temp_df['fentanyl_dose_norm'] > 0).astype(int)
non_intubated_temp_df['ketamine_binary'] = (non_intubated_temp_df['ketamine_dose_norm'] > 0).astype(int)
non_intubated_temp_df.dropna(inplace=True)
non_intubated_temp_df['ISS'] = non_intubated_temp_df['ISS'].astype(int)
fentanyl_non_intubated_iss_logit_model = sm.Logit(non_intubated_temp_df['fentanyl_binary'], sm.add_constant(non_intubated_temp_df['ISS']))
fentanyl_non_intubated_iss_logit_result = fentanyl_non_intubated_iss_logit_model.fit()
print(fentanyl_non_intubated_iss_logit_result.summary())
ketamine_non_intubated_iss_logit_model = sm.Logit(non_intubated_temp_df['ketamine_binary'], sm.add_constant(non_intubated_temp_df['ISS']))
ketamine_non_intubated_iss_logit_result = ketamine_non_intubated_iss_logit_model.fit()
print(ketamine_non_intubated_iss_logit_result.summary())

## Sex and Analgetics

In [ ]:
temp_df = df[['any_analgetics', 'Sex']]
temp_df.dropna(inplace=True)
temp_df['any_analgetics'] = temp_df['any_analgetics'].astype(int)

sex_analgetics_logit_model = sm.Logit(temp_df['any_analgetics'], sm.add_constant(temp_df['Sex']))
sex_analgetics_logit_result = sex_analgetics_logit_model.fit()
print(sex_analgetics_logit_result.summary())


## Tracheal Intubation and Survival

In [ ]:
df['1y survival'].replace(999, pd.NA, inplace=True)
temp_df = df[['Intubation (y/n)', '1y survival']].copy()
temp_df.dropna(inplace=True)
temp_df['Intubation (y/n)'] = temp_df['Intubation (y/n)'].astype(int)
temp_df['1y survival'] = temp_df['1y survival'].astype(int)

intubation_survival_logit_model = sm.Logit(temp_df['1y survival'], sm.add_constant(temp_df['Intubation (y/n)']))
intubation_survival_result = intubation_survival_logit_model.fit()
print(intubation_survival_result.summary())

## NACA and i.v. access 

In [ ]:
temp_df = df[['NACA', 'iv']].copy()
temp_df.dropna(inplace=True)
temp_df['NACA'] = temp_df['NACA'].astype(int)
temp_df['iv'] = temp_df['iv'].astype(int)
iv_naca_logit_model = sm.Logit(temp_df['iv'], sm.add_constant(temp_df['NACA']))
iv_naca_result = iv_naca_logit_model.fit()
print(iv_naca_result.summary())

## NACA and i.o. access 

In [ ]:
df['io'] = df['io'].replace(999, pd.NA)
temp_df = df[['io', 'NACA']].copy()
temp_df.dropna(inplace=True)
temp_df['io'] = temp_df['io'].astype(int)
temp_df['NACA'] = temp_df['NACA'].astype(int)
io_naca_logit_model = sm.Logit(temp_df['io'], sm.add_constant(temp_df['NACA']))
io_naca_result = io_naca_logit_model.fit()
print(io_naca_result.summary())
# print pval
print(f'IO vs NACA p-value: {io_naca_result.pvalues}')

In [ ]:
# get OR for NACA with 95% CI
import numpy as np
naca_coef = io_naca_result.params['NACA']
naca_ci_low, naca_ci_high = io_naca_result.conf_int().loc['NACA']
NACA_io_OR = np.exp(naca_coef)
NACA_io_OR_ci = (np.exp(naca_ci_low), np.exp(naca_ci_high))
print(f"NACA IO OR: {NACA_io_OR:.3f} (95% CI {NACA_io_OR_ci[0]:.3f}–{NACA_io_OR_ci[1]:.3f}, pval={io_naca_result.pvalues['NACA']})")

## NACA and ISS

In [ ]:
temp_df = df[['NACA', 'ISS']].copy()
temp_df.dropna(inplace=True)
temp_df['NACA'] = temp_df['NACA'].astype(int)
temp_df['ISS'] = temp_df['ISS'].astype(int)
# pearson correlation between NACA and ISS
naca_iss_corr, naca_iss_pval = spearmanr(temp_df['NACA'], temp_df['ISS'])
print(f"Spearman correlation between NACA and ISS: {naca_iss_corr}, p-value: {naca_iss_pval}")

In [ ]:
# plot iss vs naca
plt.figure(figsize=(10, 6))
sns.scatterplot(x='NACA', y='ISS', data=temp_df, alpha=0.5)
plt.title('In-hospital Injury Severity Score (ISS) vs NACA')
plt.xlabel('NACA')
plt.ylabel('In-hospital Injury Severity Score (ISS)')
plt.xticks(rotation=45)
plt.tight_layout()

### Over vs undertriage

In [ ]:
NACA_threshold = 5
ISS_threshold = 23

In [ ]:
temp_df = df[['NACA', 'ISS']].copy()
temp_df.dropna(inplace=True)
temp_df['NACA'] = temp_df['NACA'].astype(int)
temp_df['ISS'] = temp_df['ISS'].astype(int)

temp_df['severe_naca'] = (temp_df['NACA'] >= NACA_threshold).astype(int)
temp_df['severe_iss'] = (temp_df['ISS'] >= ISS_threshold).astype(int)

# accuracy of severe_naca to predict severe_iss
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy = accuracy_score(temp_df['severe_iss'], temp_df['severe_naca'])
conf_matrix = confusion_matrix(temp_df['severe_iss'], temp_df['severe_naca'])
class_report = classification_report(temp_df['severe_iss'], temp_df['severe_naca'])

In [ ]:
# senitivity and specificity, accuracy, ppv, npv
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"PPV: {ppv}")
print(f"NPV: {npv}")
print(f"Accuracy: {accuracy}")

In [ ]:
# number/percent of overtriaged and undertriaged patients
overtriaged = fp
undertriaged = fn
total = len(temp_df)
print(f"Overtriaged patients: {overtriaged} ({(overtriaged/total)*100:.2f}%)")
print(f"Undertriaged patients: {undertriaged} ({(undertriaged/total)*100:.2f}%)")

## MECHANISM and ISS

In [ ]:
df['Mechanism of injury'] = df['Mechanism of injury'].replace('1,6', 1)

In [ ]:
# kruskal
temp_df = df[['Mechanism of injury', 'ISS']].copy()
temp_df.dropna(inplace=True)
temp_df['Mechanism of injury'] = temp_df['Mechanism of injury'].astype(int)
from scipy.stats import kruskal

mechanism_iss_groups = [group['ISS'].values for name, group in temp_df.groupby('Mechanism of injury')]
mechanism_iss_kruskal_result = kruskal(*mechanism_iss_groups)
print(f"Kruskal-Wallis test result: H-statistic = {mechanism_iss_kruskal_result.statistic}, p-value = {mechanism_iss_kruskal_result.pvalue}")

## MECHANISM and NACA

In [ ]:
temp_df = df[['Mechanism of injury', 'NACA']].copy()
temp_df.dropna(inplace=True)
temp_df['Mechanism of injury'] = temp_df['Mechanism of injury'].astype(int)
mechanism_naca_groups = [group['NACA'].values for name, group in temp_df.groupby('Mechanism of injury')]
mechanism_naca_kruskal_result = kruskal(*mechanism_naca_groups)
print(f"Kruskal-Wallis test result for NACA: H-statistic = {mechanism_naca_kruskal_result.statistic}, p-value = {mechanism_naca_kruskal_result.pvalue}")

## Final table

In [ ]:
str_df = pd.DataFrame({
    'Association between NACA and hospital survival':[f'OR: {survival_logit_result.params["NACA"]:.2f}, p-value: {survival_logit_result.pvalues["NACA"]:.3f}'],
    'Association between ISS and analgetics':[f'OR: {iss_analgetics_logit_result.params["ISS"]:.2f}, p-value: {iss_analgetics_logit_result.pvalues["ISS"]:.3f}'],
    'Association between sex and analgetics':[f'OR: {sex_analgetics_logit_result.params["Sex"]:.2f}, p-value: {sex_analgetics_logit_result.pvalues["Sex"]:.3f}'],
    # 'Association between intubation and 1y survival':[f'OR: {intubation_survival_result.params["Intubation (y/n)"]:.2f}, p-value: {intubation_survival_result.pvalues["Intubation (y/n)"]:.3f}'],
    'Association between NACA and IV':[f'OR: {iv_naca_result.params["NACA"]:.2f}, p-value: {iv_naca_result.pvalues["NACA"]:.3f}'],
    'Association between NACA and IO':[f'OR: {io_naca_result.params["NACA"]:.2f}, p-value: {io_naca_result.pvalues["NACA"]:.3f}'],
    'Correlation between NACA and ISS':[f'Spearman Correlation: {naca_iss_corr:.2f}, p-value: {naca_iss_pval:.3f}'],
    'Association of Mechanism of Injury with ISS':[f'Kruskal-Wallis H-statistic: {mechanism_iss_kruskal_result.statistic:.2f}, p-value: {mechanism_iss_kruskal_result.pvalue:.3f}'],
    'Association of Mechanism of Injury with NACA':[f'Kruskal-Wallis H-statistic: {mechanism_naca_kruskal_result.statistic:.2f}, p-value: {mechanism_naca_kruskal_result.pvalue:.3f}'],
    'Association between fentanyl given and ISS (all patients)': [f'OR: {fentanyl_iss_logit_result.params["ISS"]:.2f}, p-value: {fentanyl_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine given and ISS (all patients)': [f'OR: {ketamine_iss_logit_result.params["ISS"]:.2f}, p-value: {ketamine_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between fentanyl dose and ISS (all patients)': [f'Coefficient: {fentanyl_iss_result.params["ISS"]:.2f}, p-value: {fentanyl_iss_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine dose and ISS (all patients)': [f'Coefficient: {ketamine_iss_result.params["ISS"]:.2f}, p-value: {ketamine_iss_result.pvalues["ISS"]:.3f}'],
    'Association between fentanyl given and ISS (intubated patients)': [f'OR: {fentanyl_intubated_iss_logit_result.params["ISS"]:.2f}, p-value: {fentanyl_intubated_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine given and ISS (intubated patients)': [f'OR: {ketamine_intubated_iss_logit_result.params["ISS"]:.2f}, p-value: {ketamine_intubated_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between fentanyl dose and ISS (intubated patients)': [f'Coefficient: {fentanyl_intubated_result.params["ISS"]:.2f}, p-value: {fentanyl_intubated_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine dose and ISS (intubated patients)': [f'Coefficient: {ketamine_intubated_result.params["ISS"]:.2f}, p-value: {ketamine_intubated_result.pvalues["ISS"]:.3f}'],
    'Association between fentanyl given and ISS (non-intubated patients)': [f'OR: {fentanyl_non_intubated_iss_logit_result.params["ISS"]:.2f}, p-value: {fentanyl_non_intubated_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine given and ISS (non-intubated patients)': [f'OR: {ketamine_non_intubated_iss_logit_result.params["ISS"]:.2f}, p-value: {ketamine_non_intubated_iss_logit_result.pvalues["ISS"]:.3f}'],
    'Association between fentanyl dose and ISS (non-intubated patients)': [f'Coefficient: {fentanyl_non_intubated_result.params["ISS"]:.2f}, p-value: {fentanyl_non_intubated_result.pvalues["ISS"]:.3f}'],
    'Association between ketamine dose and ISS (non-intubated patients)': [f'Coefficient: {ketamine_non_intubated_result.params["ISS"]:.2f}, p-value: {ketamine_non_intubated_result.pvalues["ISS"]:.3f}']
})
str_df.T

In [ ]:
# str_df.T.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/table4.csv', index=True)

In [ ]:
triage_df = pd.DataFrame({
    'NACA threshold': [NACA_threshold],
    'ISS threshold': [ISS_threshold],
    'Overtriage (FP)': [overtriaged],
    'Undertriage (FN)': [undertriaged],
    'Sensitivity': [sensitivity],
    'Specificity': [specificity],
    'PPV': [ppv],
    'NPV': [npv],
    'Accuracy': [accuracy]
}).T
triage_df

In [ ]:
# triage_df.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/triage_performance.csv', index=True)